In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="ChatBot_with_chain_Prompt_template"

In [2]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

# Build the Prompt

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

First, let's add in a system message. To do this, we will create a ChatPromptTemplate. We will utilize MessagesPlaceholder to pass all the messages in.


In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [4]:
# Build the chain
chain = prompt | model

Note that this slightly changes the input type - rather than pass in a list of messages, we are now passing in a dictionary with a messages key where that contains a list of messages.

In [5]:
from langchain_core.messages import HumanMessage

response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"})

In [6]:
response

AIMessage(content='¡Hola, Bob! ¿Cómo puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 33, 'total_tokens': 44}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-aecba663-d394-4eba-a64b-df2413af37a2-0')

# Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. 


# Define message store


In [7]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# Function to get session history
This function is expected to take in a session_id and return a Message History object. This session_id is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain


In [8]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [9]:
# Function to generate a unique session ID
import uuid

def generate_session_id() -> str:
    return str(uuid.uuid4())

In [10]:
# Generating a dynamic session ID
session_id_1 = generate_session_id()

# Define Config

We now need to create a config that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a session_id. 

In [11]:
config_1 = {"configurable": {"session_id": session_id_1}}

In [12]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config_1,
)

In [13]:
response

AIMessage(content='¡Hola, Todd! ¿En qué puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 33, 'total_tokens': 45}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_507c9469a1', 'finish_reason': 'stop', 'logprobs': None}, id='run-429801ac-2e68-4275-9eee-ee76461eb6b3-0')

In [14]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Whats my name?")], "language": "Hindi"},
    config=config_1,
)

In [15]:
response

AIMessage(content='आपका नाम टॉड है।', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 57, 'total_tokens': 65}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-3f574923-74d6-4047-b4f8-f6eab04aa7b1-0')

In [16]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What was my last question?")], "language": "English"},
    config=config_1,
)

In [17]:
response

AIMessage(content='Your last question was, "What\'s my name?"', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 79, 'total_tokens': 89}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-6905cdac-a800-4075-9b23-6665d5a3a987-0')

In [18]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Tell me my last 2 questions?")], "language": "English"},
    config=config_1,
)
response

AIMessage(content='Your last two questions were:\n\n1. "What was my last question?"\n2. "What’s my name?"', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 105, 'total_tokens': 128}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-390de423-f8c3-40aa-9dc6-1f5127409d93-0')

In [19]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Tell me my last 2 questions?")], "language": "English"},
    config=config_1,
)
response

AIMessage(content='Your last two questions were:\n\n1. "Tell me my last 2 questions?"\n2. "What was my last question?"', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 144, 'total_tokens': 170}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-1a887ba7-aeb8-417a-9c90-9909ef931f57-0')

In [20]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Tell me my last 70 questions?")], "language": "English"},
    config=config_1,
)
response

AIMessage(content='I can only provide the context of our current conversation, which includes the questions you\'ve asked so far. Here are your most recent questions:\n\n1. "Tell me my last 2 questions?"\n2. "What was my last question?"\n3. "What’s my name?"\n4. "What was my last question?"\n5. "Tell me my last 2 questions?"\n\nUnfortunately, I don\'t have the ability to retrieve a longer history of questions beyond this conversation. If you have more questions or need assistance with something else, feel free to ask!', response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 186, 'total_tokens': 296}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-0acbbda3-bd50-47c7-9dd1-358a0af074a5-0')

In [21]:
# Generating a dynamic session ID
session_id_2 = generate_session_id()
config_2 = {"configurable": {"session_id": session_id_2}}

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Whats my name?")], "language": "Hindi"},
    config=config_2,
)
response

AIMessage(content='मुझे आपका नाम नहीं पता। अगर आप चाहें तो मुझे अपना नाम बता सकते हैं!', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 33, 'total_tokens': 52}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_db4a9208a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-fa81adfd-2fad-4473-baab-6781b104bd33-0')

# Streaming final outputs
The .stream method will by default stream each key in a sequence.

Note that here only the "answer" key is streamed token-by-token, as the other components-- such as retrieval-- do not support token-level streaming.

In [24]:
stream = with_message_history.stream(
    {"messages": [HumanMessage(content="Whats my name?")], "language": "Hindi"},
    config=config_2,
)

In [26]:
for chunk in stream:
    print(chunk)

content='' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content='म' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content='ुझे' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content=' आपका' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content=' नाम' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content=' नहीं' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content=' पता' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content='।' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content=' क्या' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content=' आप' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content=' मुझे' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content=' अपना' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content=' नाम' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content=' बता' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content=' सकते' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content=' हैं' id='run-7dcec52f-f312-4eba-9baf-a15f5b2cb5f0'
content='?' id='run-7dcec52f-f